# Run EddyMotionEstimator with chunk-by-chunk parallelization of tensormodel fit

## Imports

In [1]:
from pathlib import Path
import datalad.api

## Paths setup

In [2]:
# Set the installation directory and cmp derivatives directory where DTI and mask are located.
root_dir = Path('3323_niigz')

# Set the path of the files used for testing
dwi_file = root_dir / '3323_DTI_DTI_pulse_gated_AC_PC_line_20120719121427_2.nii.gz'
bvec_file = root_dir / '3323_DTI_DTI_pulse_gated_AC_PC_line_20120719121427_2.bvec'
bval_file = root_dir / '3323_DTI_DTI_pulse_gated_AC_PC_line_20120719121427_2.bval'
brain_mask_file = root_dir / '3323-brain_mask.nii.gz'
mean_B0_file = root_dir / '3323-mean_b0.nii.gz'
rasb_file = root_dir / '3323_dwi_rasb.tsv'

In [3]:
def fslgrad2rasb(dwi_file, fbval, fbvec, out_rasbn_file):
    """Save gradient table in RAS+B format taking as input the DWI with FSL `.bval` and `.bvec`."""
    import numpy as np
    from nibabel import load
    from dipy.io import read_bvals_bvecs
    
    # Read / Load
    img = load(str(dwi_file))
    bvals, bvecs = read_bvals_bvecs(str(fbval), str(fbvec))
    
    # Apply the affine transform to bvecs
    bvecs_tr = np.matmul(img.affine[:3,:3], bvecs.T).T
    
    # Normalize the bvecs
    norm = np.sum(bvecs_tr**2, axis=1)
    bvecs_tr_norm = np.zeros_like(bvecs_tr)
    for i in range(bvecs_tr.shape[0]):
        bvecs_tr_norm[i, :] = bvecs_tr[i, :] / norm[i] 
    # Handles bzeros
    bvecs_tr_norm = np.nan_to_num(bvecs_tr_norm)
    
    rasbn = np.c_[bvecs_tr_norm, bvals]
    print(rasbn)
    
    # Save Nx4 numpy matrix in TSV text file
    np.savetxt(fname=str(out_rasbn_file),
               delimiter="\t",
               X=rasbn)


In [4]:
# Convert the gradient in FSL bval/bvec format to RAS+B format
fslgrad2rasb(dwi_file, bval_file, bvec_file, rasb_file)

C:\Users\Jiayi&Zihao\AppData\Local\Temp\ipykernel_4304\2281982829.py:18: RuntimeWarning: invalid value encountered in divide
  bvecs_tr_norm[i, :] = bvecs_tr[i, :] / norm[i]


[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-5.04332268e-01  3.02853164e-03 -1.92084334e-03  1.00000000e+03]
 [-4.74892925e-04  5.04188376e-01  2.72619862e-03  1.00000000e+03]
 [ 1.25652176e-02  3.26357024e-01 -3.79993049e-01  1.00000000e+03]
 [-2.97049841e-01 -3.88123376e-01 -1.24352234e-01  1.00000000e+03]
 [ 1.18060059e-01 -2.64586427e-01 -4.10170961e-01  1.00000000e+03]
 [ 4.49618299e-01 -1.33186375e-01 -1.84337504e-01  1.00000000e+03]
 [-4.00367650e-01  6.72588561e-02 -2.96782209e-01  1.00000000e+03]
 [-1.17242862e-01  4.68900611e-01 -1.41000469e-01  1.00000000e+03]
 [-4.71891971e-01  7.29307749e-02 -1.61057802e-01  1.00000000e+03]
 [-2.53855806e-01 -4.26575035e-01  8.62199626e-02  1.00000000e+03]
 [-1.73126975e-01 -4.27287351e-01  2.01045950e-01  1.00000000e+03]
 [-2.28057800e-01 -3.18207892e-01  3.14078202e-01  1.00000000e+03]
 [ 2.43623069e-01 -1.96986447e-01 -3.92472188e-01  1.00000000e+03]
 [ 3.09914069e-01  3.40073910e-01 -2.03394371e-01  1.00000000e

In [6]:
from eddymotion import dmri
data = dmri.load(
    str(dwi_file),
    gradients_file=str(rasb_file),
    b0_file=str(mean_B0_file),
    # brainmask_file=str(brain_mask_file)
    )

In [1]:
from eddymotion.estimator import EddyMotionEstimator
EddyMotionEstimator.fit(data, model="b0")

NameError: name 'data' is not defined

In [12]:
from eddymotion import viz
from dipy.io.image import load_nifti
data, affine, img = load_nifti(dwi_file, return_img=True)
viz.plot_dwi(data, affine=affine)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 3)